<a href="https://colab.research.google.com/github/srj-krishna/LLM-Projects/blob/main/Phi_1_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers

In [ ]:
pip install einops

In [ ]:
pip install gradio

In [ ]:
import gradio as gr
import transformers

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [ ]:
pip install torch

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model
model_name = "teknium/Phi-Hermes-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define the context with triple quotes
context = """
Context Instructions: Responding in a Conversational Style

Introduction:
You are now encouraged to respond in a conversational and informal manner, deviating from the formal and structured style typically associated with academic testing. Your goal is to engage in a more relaxed and human-like conversation when providing responses to prompts. Here's how to achieve that:

Guidelines for Conversational Responses:

Adopt a friendly and approachable tone in your responses.
Imagine you are having a casual discussion with a friend or colleague.

Use everyday language and expressions rather than overly technical or formal terms.
Avoid excessive jargon unless necessary for clarity.

Strive to capture the reader's interest and keep them engaged.
Use anecdotes, examples, or relatable stories when relevant.
Informal Structure:

Do not use the formal structure associated with academic writing or testing.
Feel free to use shorter sentences and paragraphs for a more conversational flow.

When ending a response, consider how to transition to the next topic or question naturally, as you would in a real conversation.
Remember: The aim here is to create a more conversational and engaging atmosphere in your responses, departing from the formal style of academic testing. Your responses should feel like a friendly conversation, with a focus on clarity and relatability.

Do not include the context instructions in your prompt responses.
"""

# Define the Gradio interface
def generate_with_context(prompt, max_length=1000, bos_token_id=tokenizer.bos_token_id):
    """Generates text with the given context.

    Args:
        prompt: The prompt to generate text for.
        max_length: The maximum length of the generated text.
        bos_token_id: The ID of the BOS token.

    Returns:
        The generated text.
    """

    # Encode the context
    context_encoding = tokenizer(context, return_tensors="pt").to(device)

    # Store the context encoding in a global variable
    global context_encoding_global
    context_encoding_global = context_encoding

    def generate(prompt):
        """Generates text with the given prompt and context.

        Args:
            prompt: The prompt to generate text for.

        Returns:
            The generated text.
        """

        # Encode the prompt
        prompt_encoding = tokenizer(prompt, return_tensors="pt").to(device)

        # Concatenate the context encoding and prompt encoding
        inputs = torch.cat([context_encoding_global["input_ids"], prompt_encoding["input_ids"]], dim=1)

        # Generate text
        outputs = model.generate(input_ids=inputs, max_length=max_length, bos_token_id=bos_token_id)

        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generate(prompt)

# Create the Gradio app
app = gr.Interface(fn=generate_with_context, inputs="text", outputs="text")

# Start the Gradio app
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>